## 7.1a

In [126]:
import os
import json
from pathlib import Path
import gzip
import pandas as pd
import s3fs
import hashlib
endpoint_url='https://storage.budsc.midwest-datascience.com'
current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
results_dir.mkdir(parents=True, exist_ok=True)
def read_jsonl_data():
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    with s3.open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            records = [json.loads(line) for line in f.readlines()]
    return records
def flatten_record(record):
    flat_record = dict()
    for key, value in record.items():
        if key in ['airline', 'src_airport', 'dst_airport']:
            if isinstance(value, dict):
                for child_key, child_value in value.items():
                    flat_key = '{}_{}'.format(key, child_key)
                    flat_record[flat_key] = child_value
        else:
            flat_record[key] = value
    return flat_record
def create_flattened_dataset():
    records = read_jsonl_data()
    parquet_path = results_dir.joinpath('routes-flattened.parquet')
    return pd.DataFrame.from_records([flatten_record(record) for record in records])
df = create_flattened_dataset()

In [127]:
df

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_latitude,dst_airport_longitude,dst_airport_altitude,dst_airport_timezone,dst_airport_dst,dst_airport_tz_id,dst_airport_type,dst_airport_source,codeshare,equipment
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,55.606201,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2]
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,55.606201,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2]
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,44.225101,43.081902,1054.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2]
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,55.606201,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2]
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,55.012600,82.650703,365.0,7.0,N,Asia/Krasnoyarsk,airport,OurAirports,False,[CR2]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67658,4178,Regional Express,Qantas Airways,ZL,RXA,REX,Australia,True,6334.0,Whyalla Airport,...,-34.945000,138.531006,20.0,9.5,O,Australia/Adelaide,airport,OurAirports,False,[SF3]
67659,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,4029.0,Domodedovo International Airport,...,43.061298,74.477600,2058.0,6.0,U,Asia/Bishkek,airport,OurAirports,False,[734]
67660,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,2912.0,Manas International Airport,...,55.408798,37.906300,588.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[734]
67661,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,2912.0,Manas International Airport,...,40.609001,72.793297,2927.0,6.0,U,Asia/Bishkek,airport,OurAirports,False,[734]


In [128]:
partitions = (
        ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
        ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
        ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
        ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
    )
kv_key = (
        'A', 'B', 'C-D', 'E-F',
        'G-H', 'I-J', 'K-L', 'M',
        'N', 'O-P', 'Q-R', 'S-T',
        'U', 'V', 'W-X', 'Y-Z'
    )


In [129]:
df['key'] = df.apply(lambda row: str(row.src_airport_iata).upper() + str(row.dst_airport_iata).upper()
                     + str(row.airline_iata).upper(), axis = 1) 

In [130]:
#df['kv_key'] = df['key'].str.get(0)
kv_list = []
getout = 0
for row in df['key']:
    success, counter = 0, 0
    first = row[0:1]
    for partition in partitions:
        if (partition[0] == first) or (partition[1] == first):
            kv_list.append(kv_key[counter])
            success = 1
        counter += 1
    if success == 0:
        print(row)
    

In [131]:
df['kv_key'] = kv_list

In [132]:
df

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_altitude,dst_airport_timezone,dst_airport_dst,dst_airport_tz_id,dst_airport_type,dst_airport_source,codeshare,equipment,key,kv_key
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],AERKZN2B,A
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFKZN2B,A
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,1054.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFMRV2B,A
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,411.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],CEKKZN2B,C-D
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,365.0,7.0,N,Asia/Krasnoyarsk,airport,OurAirports,False,[CR2],CEKOVB2B,C-D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67658,4178,Regional Express,Qantas Airways,ZL,RXA,REX,Australia,True,6334.0,Whyalla Airport,...,20.0,9.5,O,Australia/Adelaide,airport,OurAirports,False,[SF3],WYAADLZL,W-X
67659,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,4029.0,Domodedovo International Airport,...,2058.0,6.0,U,Asia/Bishkek,airport,OurAirports,False,[734],DMEFRUZM,C-D
67660,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,2912.0,Manas International Airport,...,588.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[734],FRUDMEZM,E-F
67661,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,2912.0,Manas International Airport,...,2927.0,6.0,U,Asia/Bishkek,airport,OurAirports,False,[734],FRUOSSZM,E-F


In [111]:
kv_dir = results_dir.joinpath('kv2')
kv_dir.mkdir(parents=True, exist_ok=True)
df.to_parquet(kv_dir, partition_cols = ['kv_key'])


## 7.1b

In [133]:
import hashlib
def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return m.hexdigest()

df['hashed'] = df.apply(lambda row: hash_key(row.key), axis = 1)

df['hash_key'] = df.apply(lambda row: row.hashed[0].upper(), axis = 1)

In [134]:
df

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_dst,dst_airport_tz_id,dst_airport_type,dst_airport_source,codeshare,equipment,key,kv_key,hashed,hash_key
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,N,Europe/Moscow,airport,OurAirports,False,[CR2],AERKZN2B,A,652cdec02010381f175efe499e070c8cbaac1522bac59a...,6
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFKZN2B,A,9eea5dd88177f8d835b2bb9cb27fb01268122b635b241a...,9
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,N,Europe/Moscow,airport,OurAirports,False,[CR2],ASFMRV2B,A,161143856af25bd4475f62c80c19f68936a139f653c1d3...,1
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,N,Europe/Moscow,airport,OurAirports,False,[CR2],CEKKZN2B,C-D,39aa99e6ae2757341bede9584473906ef1089e30820c90...,3
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,N,Asia/Krasnoyarsk,airport,OurAirports,False,[CR2],CEKOVB2B,C-D,143b3389bce68eea3a13ac26a9c76c1fa583ec2bd26ea8...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67658,4178,Regional Express,Qantas Airways,ZL,RXA,REX,Australia,True,6334.0,Whyalla Airport,...,O,Australia/Adelaide,airport,OurAirports,False,[SF3],WYAADLZL,W-X,f31527be84c36208c05cac57dfac8a46b48a87dda151f8...,F
67659,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,4029.0,Domodedovo International Airport,...,U,Asia/Bishkek,airport,OurAirports,False,[734],DMEFRUZM,C-D,880fc35ca283ad034c90becc4e331b72ee894b9eb69f76...,8
67660,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,2912.0,Manas International Airport,...,N,Europe/Moscow,airport,OurAirports,False,[734],FRUDMEZM,E-F,e976939986fbf947bb9318018cef717c0b34dff91e5e67...,E
67661,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,2912.0,Manas International Airport,...,U,Asia/Bishkek,airport,OurAirports,False,[734],FRUOSSZM,E-F,8b0c0b835a58a4250e020d51ec2a896e4ef3f5c3543b8e...,8


In [114]:
hash_dir = results_dir.joinpath('hash')
hash_dir.mkdir(parents=True, exist_ok=True)
df.to_parquet(hash_dir, partition_cols = ['hash_key'])

## 7.1c

In [135]:
import pygeohash

def datacenter_search(latitude, longitude, west, central, east):

    target = pygeohash.encode(latitude, longitude, precision = 5)
    distancew = pygeohash.geohash_haversine_distance(target, west)
    distancec = pygeohash.geohash_haversine_distance(target, central)
    distancee = pygeohash.geohash_haversine_distance(target, east)
    
    if (distancew < distancec) and (distancew < distancee):
        return 'west'
    elif (distancec < distancew) and (distancec < distancee):
        return 'central'
    else:
        return 'east'

    

In [136]:
west = pygeohash.encode(45.5945645, 121.1786823, precision = 5)
central = pygeohash.encode(41.1544433, 96.0422378, precision = 5)
east = pygeohash.encode(39.08344, 77.6497145, precision = 5)

df['location'] = df.apply(
    lambda row: datacenter_search(row.src_airport_latitude,row.src_airport_longitude,
                                  west, central, east), axis = 1)

In [138]:
geo_dir = results_dir.joinpath('geo')
geo_dir.mkdir(parents=True, exist_ok=True)
df.to_parquet(geo_dir, partition_cols = ['location'])

In [137]:
df

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_tz_id,dst_airport_type,dst_airport_source,codeshare,equipment,key,kv_key,hashed,hash_key,location
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,Europe/Moscow,airport,OurAirports,False,[CR2],AERKZN2B,A,652cdec02010381f175efe499e070c8cbaac1522bac59a...,6,east
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,Europe/Moscow,airport,OurAirports,False,[CR2],ASFKZN2B,A,9eea5dd88177f8d835b2bb9cb27fb01268122b635b241a...,9,east
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,Europe/Moscow,airport,OurAirports,False,[CR2],ASFMRV2B,A,161143856af25bd4475f62c80c19f68936a139f653c1d3...,1,east
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,Europe/Moscow,airport,OurAirports,False,[CR2],CEKKZN2B,C-D,39aa99e6ae2757341bede9584473906ef1089e30820c90...,3,east
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,Asia/Krasnoyarsk,airport,OurAirports,False,[CR2],CEKOVB2B,C-D,143b3389bce68eea3a13ac26a9c76c1fa583ec2bd26ea8...,1,east
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67658,4178,Regional Express,Qantas Airways,ZL,RXA,REX,Australia,True,6334.0,Whyalla Airport,...,Australia/Adelaide,airport,OurAirports,False,[SF3],WYAADLZL,W-X,f31527be84c36208c05cac57dfac8a46b48a87dda151f8...,F,west
67659,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,4029.0,Domodedovo International Airport,...,Asia/Bishkek,airport,OurAirports,False,[734],DMEFRUZM,C-D,880fc35ca283ad034c90becc4e331b72ee894b9eb69f76...,8,east
67660,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,2912.0,Manas International Airport,...,Europe/Moscow,airport,OurAirports,False,[734],FRUDMEZM,E-F,e976939986fbf947bb9318018cef717c0b34dff91e5e67...,E,east
67661,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,2912.0,Manas International Airport,...,Asia/Bishkek,airport,OurAirports,False,[734],FRUOSSZM,E-F,8b0c0b835a58a4250e020d51ec2a896e4ef3f5c3543b8e...,8,east


## 7.1d

In [175]:
def balance_partitions(keys, num_partitions):
    partitions = []
    keys.sort()                              # Sorts the keys.
    batch = int(len(keys) / num_partitions)  # Figures out how many keys per partition, rounded down.
    remainder = len(keys) % num_partitions   # Figures out the remainder to distribute the keys that don't fit.
    #print(batch)
    #print(remainder)
    batch_counter = 0
    batch_list = []
    for x in keys:
        #print(x)
        #print(batch_counter)
        if batch_counter < batch:           
            batch_list.append(x)
        elif remainder > 0:                   # Allocates a single key to the first x partitions, where x = remainder.
            batch_list.append(x)
            remainder -= 1
        else:
            #print(batch_list)
            partitions.append(batch_list)      # When the keys in the batch hit the max bax size, add that list to partitions
            batch_counter = 0                  # and reset.
            batch_list = [x]
        batch_counter += 1 
    partitions.append(batch_list)              # Add the last batch to the partition.
    return partitions

keys = ['A1', 'A3', 'B6', 'R4', 'B1', 'X4', 'A2', 'H2', 'V4', 'S3']
balance_partitions(keys, 3)

[['A1', 'A2', 'A3', 'B1'], ['B6', 'H2', 'R4'], ['S3', 'V4', 'X4']]

In [179]:
wdw = ['Astro Orbiter',                      # To scale up, we used a list of every ride at WDW, sorted by park initially.
'The Barnstormer',
'Big Thunder Mountain Railroad',
'Buzz Lightyear\'s Space Ranger Spin',
'Casey Jr, Splash \'N\' Soak Station',
'Cinderella Castle',
'Country Bear Jamboree',
'Dumbo the Flying Elephant',
'Enchanted Tales with Belle',
'Enchanted Tiki Room',
'Frontierland Shootin\' Arcade',
'The Hall of Presidents',
'Haunted Mansion',
'It\'s a Small World',
'Jungle Cruise',
'Liberty Square Riverboat',
'Mad Tea Party',
'The Magic Carpets of Aladdin',
'Main Street Vehicles – Themed Area',
'The Many Adventures of Winnie the Pooh',
'Mickey\'s PhilharMagic',
'Mickey\'s Royal Friendship Faire stage show',
'Monsters, Inc. Laugh Floor',
'Peter Pan\'s Flight',
'A Pirate\'s Adventure ~ Treasures of the Seven Seas',
'Pirates of the Caribbean',
'Prince Charming Regal Carrousel',
'Seven Dwarfs Mine Train',
'Sorcerers of the Magic Kingdom[1]',
'Space Mountain',
'Splash Mountain',
'Swiss Family Treehouse',
'Tom Sawyer Island',
'Tomorrowland Speedway',
'Tomorrowland Transit Authority PeopleMover',
'Under the Sea ~ Journey of The Little Mermaid',
'Walt Disney World Railroad',
'Walt Disney\'s Carousel of Progress',
'Advanced Training Lab',
'The American Adventure',
'American Heritage Gallery',
'Awesome Planet',
'Beauty and the Beast: Sing-Along',
'Bijutsu-kan Gallery',
'Canada: Far and Wide',
'Disney & Pixar Short Film Festival',
'Meet and Greet Anna and Elsa from Frozen',
'Gallery of Arts and History',
'Gran Fiesta Tour Starring The Three Caballeros',
'House of the Whispering Willows',
'ImageWorks: The What-If Labs',
'Impressions de France',
'Journey into Imagination with Figment',
'Kidcot Fun Stops',
'Living with the Land',
'Frozen Ever After',
'Mexico Folk Art Gallery',
'Mission: SPACE',
'Project Tomorrow: Inventing the Wonders of the Future',
'Reflections of China',
'SeaBase',
'The Seas with Nemo & Friends',
'Soarin\'',
'Spaceship Earth',
'Stave Church Gallery',
'Test Track',
'Turtle Talk with Crush',
'Alien Swirling Saucers',
'Beauty and the Beast Live on Stage',
'Celebrity Spotlight',
'Disney Junior Play & Dance!',
'Fantasmic!',
'For the First Time in Forever: A Frozen Sing-Along Celebration',
'Mickey and Minnie\'s Runaway Railway',
'Indiana Jones Epic Stunt Spectacular!',
'Jedi Training: Trials of the Temple',
'Lightning McQueen\'s Racing Academy',
'Mickey Short Theater',
'Millennium Falcon Smugglers Run',
'Muppet*Vision 3D',
'Rise of the Resistance',
'Rock \'n\' Roller Coaster Starring Aerosmith',
'Slinky Dog Dash',
'Star Wars Launch Bay',
'Star Tours: The Adventures Continue',
'Star Wars: A Galactic Spectacular',
'Toy Story Mania!',
'The Twilight Zone Tower of Terror',
'Voyage of the Little Mermaid',
'Walt Disney Presents',
'Affection Section',
'Avatar Flight of Passage',
'The Boneyard',
'Conservation Station',
'Cretaceous Trail',
'Dinosaur',
'Dino Sue',
'Discovery Island Trails',
'Expedition Everest: Legend of the Forbidden Mountain',
'Fossil Fun Games',
'Finding Nemo - The Musical',
'Habitat Habit!',
'It\'s Tough to be a Bug!',
'Kali River Rapids',
'Kilimanjaro Safaris',
'Maharajah Jungle Trek',
'Na\'vi River Journey',
'The Oasis Exhibits',
'Pangani Forest Exploration Trail',
'Tree of Life',
'TriceraTop Spin',
'UP! A Great Bird Adventure',
'Wild Africa Trek',
'Wilderness Explorers',
'Wildlife Express Train',
'Festival of the Lion King']

In [180]:
wdw

['Astro Orbiter',
 'The Barnstormer',
 'Big Thunder Mountain Railroad',
 "Buzz Lightyear's Space Ranger Spin",
 "Casey Jr, Splash 'N' Soak Station",
 'Cinderella Castle',
 'Country Bear Jamboree',
 'Dumbo the Flying Elephant',
 'Enchanted Tales with Belle',
 'Enchanted Tiki Room',
 "Frontierland Shootin' Arcade",
 'The Hall of Presidents',
 'Haunted Mansion',
 "It's a Small World",
 'Jungle Cruise',
 'Liberty Square Riverboat',
 'Mad Tea Party',
 'The Magic Carpets of Aladdin',
 'Main Street Vehicles\xa0– Themed Area',
 'The Many Adventures of Winnie the Pooh',
 "Mickey's PhilharMagic",
 "Mickey's Royal Friendship Faire stage show",
 'Monsters, Inc. Laugh Floor',
 "Peter Pan's Flight",
 "A Pirate's Adventure ~ Treasures of the Seven Seas",
 'Pirates of the Caribbean',
 'Prince Charming Regal Carrousel',
 'Seven Dwarfs Mine Train',
 'Sorcerers of the Magic Kingdom[1]',
 'Space Mountain',
 'Splash Mountain',
 'Swiss Family Treehouse',
 'Tom Sawyer Island',
 'Tomorrowland Speedway',
 'Tom

In [181]:
balance_partitions(wdw, 3)

[["A Pirate's Adventure ~ Treasures of the Seven Seas",
  'Advanced Training Lab',
  'Affection Section',
  'Alien Swirling Saucers',
  'American Heritage Gallery',
  'Astro Orbiter',
  'Avatar\xa0Flight of Passage',
  'Awesome Planet',
  'Beauty and the Beast Live on Stage',
  'Beauty and the Beast: Sing-Along',
  'Big Thunder Mountain Railroad',
  'Bijutsu-kan Gallery',
  "Buzz Lightyear's Space Ranger Spin",
  'Canada: Far and Wide',
  "Casey Jr, Splash 'N' Soak Station",
  'Celebrity Spotlight',
  'Cinderella Castle',
  'Conservation Station',
  'Country Bear Jamboree',
  'Cretaceous Trail',
  'Dino Sue',
  'Dinosaur',
  'Discovery Island\xa0Trails',
  'Disney & Pixar Short Film Festival',
  'Disney Junior Play & Dance!',
  'Dumbo the Flying Elephant',
  'Enchanted Tales with Belle',
  'Enchanted Tiki Room',
  'Expedition Everest: Legend of the Forbidden Mountain',
  'Fantasmic!',
  'Festival of the Lion King',
  'Finding Nemo - The Musical',
  'For the First Time in Forever: A Fro

In [182]:
balance_partitions(wdw, 7)

[["A Pirate's Adventure ~ Treasures of the Seven Seas",
  'Advanced Training Lab',
  'Affection Section',
  'Alien Swirling Saucers',
  'American Heritage Gallery',
  'Astro Orbiter',
  'Avatar\xa0Flight of Passage',
  'Awesome Planet',
  'Beauty and the Beast Live on Stage',
  'Beauty and the Beast: Sing-Along',
  'Big Thunder Mountain Railroad',
  'Bijutsu-kan Gallery',
  "Buzz Lightyear's Space Ranger Spin",
  'Canada: Far and Wide',
  "Casey Jr, Splash 'N' Soak Station",
  'Celebrity Spotlight',
  'Cinderella Castle',
  'Conservation Station',
  'Country Bear Jamboree',
  'Cretaceous Trail'],
 ['Dino Sue',
  'Dinosaur',
  'Discovery Island\xa0Trails',
  'Disney & Pixar Short Film Festival',
  'Disney Junior Play & Dance!',
  'Dumbo the Flying Elephant',
  'Enchanted Tales with Belle',
  'Enchanted Tiki Room',
  'Expedition Everest: Legend of the Forbidden Mountain',
  'Fantasmic!',
  'Festival of the Lion King',
  'Finding Nemo - The Musical',
  'For the First Time in Forever: A Fr